In [2]:
from google.colab import drive

In [3]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/MyDrive/Colab Notebooks/stdcode2024/Session07 - 머신러닝

/content/gdrive/MyDrive/Colab Notebooks/stdcode2024/Session07 - 머신러닝


### cars.data....분석해보기

In [1]:
import pandas as pd

In [ ]:
### Step1: 데이터를 바꾸더라도... 혼자 할 수 있는힘
### Step2: 분석 시 카테고리에 대한 이해를!!!
### Step3: 모델을 바꾸는거

In [15]:
secData = pd.read_csv("../dataset/kopo_decision_tree_all_new.csv")

### 1. 데이터 타입 통합 및 특성 숫자컬럼 추가

In [20]:
## 데이터 타입 통합 skip

In [21]:
from sklearn.preprocessing import LabelEncoder

In [22]:
ynLabel = LabelEncoder()

In [24]:
secData["LE_HOLIDAY"] = ynLabel.fit_transform( secData.HOLIDAY )
secData["LE_PROMOTION"] = ynLabel.fit_transform( secData.PROMOTION )

### 2. 특성 선정 / 데이터 분리

In [26]:
corrDf = secData.corr(numeric_only=True)

In [27]:
stdCorr = 0.5

### 그룹바이

In [34]:
groupKey = ["REGIONID","ITEM"]

In [38]:
### 머신러닝의 한계!! 가 있다
### 머신러닝은 그룹별로 건수가 많지 않으면... 아에 머신러닝 불가능!

### 머신러닝이 할 수 있는건 어떻게? 나누는지?

In [41]:
groupData = secData.groupby(by=groupKey)[["QTY"]].agg("count").reset_index()

In [44]:
groupData = groupData.rename(columns={"QTY": "QTY_COUNT"})

In [47]:
mergedData = pd.merge(left = secData,
         right =  groupData,
         on = groupKey,
         how = "left")

In [59]:
# 1년기준 52주 데이터 가 없으면 머신러닝 안함!
maxKnob = 104

In [60]:
mergedData

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,LE_HOLIDAY,LE_PROMOTION,QTY_COUNT
0,A01,PG01,P01,ITEM001,201538,2015,38,1,N,4,N,0.0,0,0,3
1,A01,PG01,P01,ITEM001,201548,2015,48,1,Y,0,N,0.0,1,0,3
2,A01,PG01,P01,ITEM001,201549,2015,49,2,Y,0,N,0.0,1,0,3
3,A01,PG01,P01,ITEM002,201526,2015,26,1,Y,1,N,0.0,1,0,2
4,A01,PG01,P01,ITEM002,201532,2015,32,1,N,4,N,0.0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32410,A01,PG05,P16,ITEM491,201503,2015,3,1,N,4,N,0.0,0,0,5
32411,A01,PG05,P16,ITEM491,201504,2015,4,1,Y,2,N,0.0,1,0,5
32412,A01,PG05,P16,ITEM491,201505,2015,5,2,N,4,N,0.0,0,0,5
32413,A01,PG05,P16,ITEM491,201506,2015,6,1,N,4,N,0.0,0,0,5


In [56]:
nomlData = mergedData.loc[ mergedData.QTY_COUNT < maxKnob ]

In [61]:
## 할수 있는거를 알려줘야한다!
mlData = mergedData.loc[ mergedData.QTY_COUNT >= maxKnob ]

In [64]:
mlGroupData = mlData.groupby(by = groupKey )

In [178]:
def mlFunc( eachGroup ):
    regionIdData = eachGroup.REGIONID.values[0]
    itemData = eachGroup.ITEM.values[0]
    print(regionIdData, itemData)
    # 그룹바이 머신러닝!!
    # 그룹데이터중에 한개를 추출한다
    # 한개의 그룹에대해서 머신러닝 로직을 구현한다.
    # 해당 로직을 함수로 설정한다
    # 각 그룹별 머신러닝을 구동한다
    # mlGroupData = mlData.groupby(by = groupKey ).apply ( 해당함수)
    # 디버깅 코드
    #eachGroup = mlGroupData.get_group(   list( mlGroupData.groups )[100] )
    ### 2. 특성 선정 / 데이터 분리
    corrDf = eachGroup.corr(numeric_only=True)
    label = ["QTY"]
    stdRatio = [0.5, 0.4, 0.3, 0.2, 0.1]
    import numpy as np
    stdRatio =  list( np.array( range( 5, 1, -1) ) / 10 )
    stdRatio
    for i in range(0, len(stdRatio)):
        features = list ( corrDf.loc[ abs(corrDf.QTY) >= stdRatio[i] ].index )
        featureLen = len(features)
        if featureLen <=1:
            continue
        elif featureLen >= 2:
            break
        else:
            print("exception")
    simulatedRatio = stdRatio[i]
    features = list( set(features) - set(label) )
    from sklearn.model_selection import train_test_split
    trainData, testData = train_test_split( eachGroup, test_size= 0.2, random_state=1 )
    trainingDataFeatures = trainData.loc[:, features]
    trainingDataLabel = trainData.loc[:, label]
    testDataFeatures = testData.loc[:, features]
    testDataLabel = testData.loc[:, label]
    testDataAll = testData.loc[:, ]
    print(trainingDataFeatures.shape)
    print(trainingDataLabel.shape)
    print(testDataFeatures.shape)
    print(testDataLabel.shape)
    print(testDataAll.shape)
    ### 3. 모델 적용
    from sklearn import tree
    dtModel = tree.DecisionTreeRegressor(random_state=10)
    fittedModel = dtModel.fit(X=trainingDataFeatures, y=trainingDataLabel)
    ### 4. 예측
    predictValue = fittedModel.predict(testDataFeatures)
    testDataLabel["DT_PREDICT"] = predictValue
    testDataAll["DT_PREDICT"] = predictValue
    testDataAll["STDRATIO"] = simulatedRatio

    return testDataAll

In [179]:
mlData.groupby(by = groupKey ).apply(mlFunc)

A01 ITEM013
(104, 2)
(104, 1)
(26, 2)
(26, 1)
(26, 15)
A01 ITEM014
(111, 3)
(111, 1)
(28, 3)
(28, 1)
(28, 15)
A01 ITEM015
(112, 3)
(112, 1)
(28, 3)
(28, 1)
(28, 15)
A01 ITEM016
(111, 3)
(111, 1)
(28, 3)
(28, 1)
(28, 15)
A01 ITEM018
(93, 1)
(93, 1)
(24, 1)
(24, 1)
(24, 15)
A01 ITEM020
(97, 3)
(97, 1)
(25, 3)
(25, 1)
(25, 15)
A01 ITEM034
(88, 2)
(88, 1)
(23, 2)
(23, 1)
(23, 15)
A01 ITEM042
(116, 3)
(116, 1)
(29, 3)
(29, 1)
(29, 15)
A01 ITEM043
(116, 1)
(116, 1)
(30, 1)
(30, 1)
(30, 15)
A01 ITEM044
(116, 2)
(116, 1)
(30, 2)
(30, 1)
(30, 15)
A01 ITEM051
(116, 1)
(116, 1)
(30, 1)
(30, 1)
(30, 15)
A01 ITEM052
(88, 1)
(88, 1)
(22, 1)
(22, 1)
(22, 15)
A01 ITEM053
(116, 2)
(116, 1)
(30, 2)
(30, 1)
(30, 15)
A01 ITEM054
(116, 2)
(116, 1)
(30, 2)
(30, 1)
(30, 15)
A01 ITEM056
(116, 2)
(116, 1)
(30, 2)
(30, 1)
(30, 15)
A01 ITEM058
(85, 3)
(85, 1)
(22, 3)
(22, 1)
(22, 15)
A01 ITEM059
(113, 1)
(113, 1)
(29, 1)
(29, 1)
(29, 15)
A01 ITEM061
(116, 2)
(116, 1)
(29, 2)
(29, 1)
(29, 15)
A01 ITEM084
(116, 1)

REGIONID PRODUCTGROUP PRODUCT     ITEM  YEARWEEK  YEAR  \
REGIONID ITEM                                                                   
A01      ITEM013 527        A01         PG01     P01  ITEM013    201710  2017   
                 524        A01         PG01     P01  ITEM013    201706  2017   
                 454        A01         PG01     P01  ITEM013    201536  2015   
                 543        A01         PG01     P01  ITEM013    201729  2017   
                 472        A01         PG01     P01  ITEM013    201601  2016   
...                         ...          ...     ...      ...       ...   ...   
         ITEM465 31446      A01         PG05     P16  ITEM465    201641  2016   
                 31441      A01         PG05     P16  ITEM465    201636  2016   
                 31395      A01         PG05     P16  ITEM465    201543  2015   
                 31426      A01         PG05     P16  ITEM465    201621  2016   
                 31370      A01         PG05     P16  ITEM465    201518  2015   

                        WEEK   QTY HOLIDAY  HCLUS PROMOTION  PRO_PERCENT  \
REGIONID ITEM                                                              
A01      ITEM013 527      10     1       N      4         Y     0.125328   
                 524       6     1       N      4         Y     0.187805   
                 454      36     8       Y      1         Y     0.187805   
                 543      29     1       Y      1         Y     0.406754   
                 472       1     5       N      4         Y     0.187805   
...                      ...   ...     ...    ...       ...          ...   
         ITEM465 31446    41   879       Y      2         Y     0.308385   
                 31441    36  1757       Y      1         Y     0.333166   
                 31395    43   306       N      4         N     0.000000   
                 31426    21    80       Y      1         N     0.000000   
                 31370    18   286       N      4         N     0.000000   

                        LE_HOLIDAY  LE_PROMOTION  QTY_COUNT   DT_PREDICT  \
REGIONID ITEM                                                              
A01      ITEM013 527             0             1        130     4.000000   
                 524             0             1        130     4.000000   
                 454             1             1        130     6.000000   
                 543             1             1        130     3.000000   
                 472             0             1        130     6.000000   
...                            ...           ...        ...          ...   
         ITEM465 31446           1             1        110  1049.642857   
                 31441           1             1        110  1043.000000   
                 31395           0             0        110   413.903226   
                 31426           1             0        110   413.903226   
                 31370           0             0        110   413.903226   

                        STDRATIO  
REGIONID ITEM                     
A01      ITEM013 527         0.5  
                 524         0.5  
                 454         0.5  
                 543         0.5  
                 472         0.5  
...                          ...  
         ITEM465 31446       0.5  
                 31441       0.5  
                 31395       0.5  
                 31426       0.5  
                 31370       0.5  

[3570 rows x 17 columns]

# 그룹바이 함수 디버깅

In [155]:
# 그룹바이 머신러닝!!
# 그룹데이터중에 한개를 추출한다
# 한개의 그룹에대해서 머신러닝 로직을 구현한다.
# 해당 로직을 함수로 설정한다
# 각 그룹별 머신러닝을 구동한다
# mlGroupData = mlData.groupby(by = groupKey ).apply ( 해당함수)
# 디버깅 코드
eachGroup = mlGroupData.get_group(   list( mlGroupData.groups )[100] )



In [171]:
eachGroup = mlData.loc[ (mlData.REGIONID =="A01") &
            (mlData.ITEM =="ITEM013")  ]

In [172]:

### 2. 특성 선정 / 데이터 분리
corrDf = eachGroup.corr(numeric_only=True)
label = ["QTY"]
stdRatio = [0.5, 0.4, 0.3, 0.2, 0.1]
import numpy as np
stdRatio =  list( np.array( range( 5, 1, -1) ) / 10 )
stdRatio


[0.5, 0.4, 0.3, 0.2]

In [176]:
for i in range(0, len(stdRatio)):
    features = list ( corrDf.loc[ abs(corrDf.QTY) >= stdRatio[i] ].index )
    featureLen = len(features)
    if featureLen <=1:
        continue
    elif featureLen >= 2:
        break
    else:
        print("exception")


In [177]:
features

['YEARWEEK', 'YEAR', 'QTY']

In [ ]:
simulatedRatio = stdRatio[i]
features = list( set(features) - set(label) )
from sklearn.model_selection import train_test_split
trainData, testData = train_test_split( eachGroup, test_size= 0.2, random_state=1 )
trainingDataFeatures = trainData.loc[:, features]
trainingDataLabel = trainData.loc[:, label]
testDataFeatures = testData.loc[:, features]
testDataLabel = testData.loc[:, label]
testDataAll = testData.loc[:, ]
print(trainingDataFeatures.shape)
print(trainingDataLabel.shape)
print(testDataFeatures.shape)
print(testDataLabel.shape)
print(testDataAll.shape)
### 3. 모델 적용
from sklearn import tree
dtModel = tree.DecisionTreeRegressor(random_state=10)
fittedModel = dtModel.fit(X=trainingDataFeatures, y=trainingDataLabel)
### 4. 예측
predictValue = fittedModel.predict(testDataFeatures)
testDataLabel["DT_PREDICT"] = predictValue
testDataAll["DT_PREDICT"] = predictValue
testDataAll["STDRATIO"] = simulatedRatio